In [1]:
from astropy import units as u
from astropy.io import fits
from scipy import interpolate
import aplpy
#import ccdproc
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import statistics

In [1]:
def func(file):
    
    #Opens the .fits file
    hdu = fits.open(file)
    
    #Variable for the image of the fits
    im = hdu[0].data
    
    #Variable for the header of the fits
    head = hdu[0].header
    
    #The x-position and y-position at which Channel A begins and ends
    xminA,xmaxA = head['S_EFMN11'],head['S_EFMX11']
    yminA,ymaxA = head['S_EFMN12'],head['S_EFMX12']
    chA = im[yminA:ymaxA,xminA:xmaxA]
    
    #The x-position and y-position at which Channel B begins and ends
    xminB,xmaxB = head['S_EFMN21'],head['S_EFMX21']
    yminB,ymaxB = head['S_EFMN22'],head['S_EFMX22']
    chB = im[yminB:ymaxB,xminB:xmaxB]
    
    #The x-position and y-position at which Channel C begins and ends
    xminC,xmaxC = head['S_EFMN31'],head['S_EFMX31']
    yminC,ymaxC = head['S_EFMN32'],head['S_EFMX32']
    chC = im[yminC:ymaxC,xminC:xmaxC]
    
    #The x-position and y-position at which Channel D begins and ends
    xminD,xmaxD = head['S_EFMN41'],head['S_EFMX41']
    yminD,ymaxD = head['S_EFMN42'],head['S_EFMX42']
    chD = im[yminD:ymaxD,xminD:xmaxD]
    
    #Channel A
    
    #Re-positioning to account for .fits' offset
    head['CRPIX2'] = head['CRPIX2'] - yminA
    
    #Boundaries of overscan columns
    ovs_l,ovs_r = head['S_OSMN11'],head['S_OSMX11']
    
    #Creates an empty vector of the dimensions of the y-axis of the image
    pry = np.empty(head['NAXIS2'])
    
    #Creates an integer vector from 0 to NAXIS2
    z = np.arange(head['NAXIS2'])
    
    #Runs through all the rows in the overscan column, averaging the value of the bias for each row
    #Whilst taking into account bad pixels which are set to NaN
    for i in range(head['NAXIS2']):
        pry[i] = np.nanmean(im[i,ovs_l:ovs_r]) #np.nanmean is always better than np.mean 'cause it accounts the NaNs
        if pry[i] > 60000:
            pry[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the rows
    meanpry = np.nanmean(pry)
    stdy = np.nanstd(pry)
    
    #Replaces all values higher/lower than 2 standard deviations from the mean by NaN
    pry[pry > (meanpry + 2*stdy)] = np.nan
    pry[pry < (meanpry - 2*stdy)] = np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indy = np.isfinite(pry)
    
    #Calculates the parameters for the polyfit function
    py = np.polyfit(z[indy],pry[indy],5)
    
    #Builds the poly function automatically
    y = np.polyval(py,z)
    
    #Scatter so not to connect the dots; Plot to show the line;
    #Sets the color and the y-axis limit
    #plt.figure(1)
    #plt.scatter(z,pry)
    #plt.plot(z,y,color='red')
    #plt.ylim(meanpry-3*stdy,meanpry+3*stdy)
    
    #The effective x-dimension of the Channel
    effx = xmaxA - xminA
    
    #Replicate the polyfit into an image of effx columns
    bias = np.tile(y,(effx,1))
    
    #Bias must be transposed to match dimensions
    bias = np.transpose(bias)
    
    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelA = im[:,xminA:xmaxA]
    
    #Applies the overscan reduction due to the overscan columns
    imredbA1 = channelA - bias
    
    #Boundaries of overscan rows
    ovs_d,ovs_up = head['S_OSMN12'],head['S_OSMX12']
    
    #Offset due to some errors encountered
    ovs_u = ovs_up - 2
    
    #Creates an empty vector of dimension effx
    prx = np.empty(effx)
    
    #Creates an integer vector from xminA to xmaxA
    x = np.arange(xminA,xmaxA)
    
    #Template matrix for the overscan rows, which have already been affected
    #By the reduction of the overscan columns
    tmp = imredbA1[ovs_d:ovs_u,:]
    
    #Runs through all the columns in the overscan row, averaging the value of the bias for each column
    #Whilst taking into account bad pixels which are set to NaN
    for i in range((effx-1)):
        prx[i] = np.nanmean(imredbA1[ovs_d:ovs_u,i])
        if prx[i] > 40000:
            prx[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the columns
    meanprx = np.nanmean(prx)
    stdx = np.nanstd(prx)

    #Plots a histogram with the values of the mean in each of the columns to check for any remaining bad pixels
    #plt.figure(2)
    #plt.hist(prx,bins=100)
    
    #Replaces all values higher/lower than 2 standard deviations from the mean by NaN
    prx[prx > (meanprx + 2*stdx)] = np.nan
    prx[prx < (meanprx - 2*stdx)] = np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indx = np.isfinite(prx)
    
    #Calculates the parameters for the polyfit function
    px = np.polyfit(x[indx],prx[indx],5)
    
    #Builds the poly function automatically
    u = np.polyval(px,x)
    
    #Scatter so not to connect the dots; Plot to show the line;
    #Sets the color and the y-axis limit
    #plt.figure(3)
    #plt.scatter(x,prx)
    #plt.plot(x,u,color='red')
    #plt.ylim(meanprx-stdx,meanprx+stdx)
    
    #Transposes the template matrix to match dimensions
    tmp = np.transpose(tmp)
    
    #Plots the values of the bias counts in the overscan row
    #Sets the color and the y-axis limit
    #plt.figure(4)
    #plt.plot(x,tmp,color='red')
    #plt.ylim(7, 35)
    
    #The effective y-dimension of the Channel
    effy = ymaxA - yminA
    
    #Replicate the polyfit into an image of effx columns
    bias = np.tile(u,(effy,1))

    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelA = imredbA1[yminA:ymaxA,:]
    
    #Applies the overscan reduction due to the residual noise left over in the overscan rows
    imredbA2 = channelA - bias
    
    #Boundaries of overscan columns
    ovs_l,ovs_r = head['S_OSMN21'],head['S_OSMX21']
    
    #Creates an empty vector of dimension NAXIS2
    pry = np.empty(head['NAXIS2'])
    
    #Creates an integer vector from 0 to NAXIS2
    z = np.arange(head['NAXIS2'])
    
    #Runs through all the rows in the overscan column, averaging the value of the bias for each row
    #Whilst taking into account bad pixels which are set to NaN
    for i in range(head['NAXIS2']):
        pry[i] = np.nanmean(im[i,ovs_l:ovs_r]) #np.nanmean is always better than np.mean 'cause it accounts the NaNs
        if pry[i] > 60000:
            pry[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the rows
    meanpry = np.nanmean(pry)
    stdy = np.nanstd(pry)
    
    #Replaces all values higher/lower than 1 Stds (different from channel A) from the mean by a NaN
    pry[pry > (meanpry + stdy)] = np.nan
    pry[pry < (meanpry - stdy)] = np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indy = np.isfinite(pry)
    
    #Calculates the parameters for the polyfit function
    py = np.polyfit(z[indy],pry[indy],5)
    
    #Builds the poly function automatically
    y = np.polyval(py,z)
    
    #Scatter so not to connect the dots; Plot to show the line; Show() is for when using the command line
    #plt.figure(5)
    #plt.scatter(z,pry)
    #plt.plot(z,y,color='red')
    #plt.ylim(meanpry-3*stdy,meanpry+3*stdy)
    
    #The effective x-dimension of the Channel
    effx = xmaxB - xminB
    
    #Replicate the polyfit into an image of effx columns
    bias = np.tile(y,(effx,1))
    
    #Bias must be transposed to match dimensions
    bias = np.transpose(bias)

    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelB = im[:,xminB:xmaxB]
    
    #Applies the overscan reduction due to the overscan columns
    imredbB1 = channelB - bias
    
    #Boundaries of overscan rows
    ovs_d,ovs_up = head['S_OSMN22'],head['S_OSMX22']
    
    #Offset due to some errors encountered
    ovs_u = ovs_up - 2
    
    #Creates an empty vector of dimension effx
    prx = np.empty(effx)
    
    #Creates an integer vector from xminA to xmaxA
    x = np.arange(xminB,xmaxB)
    
    #Template matrix for the overscan rows, which have already been affected
    #By the reduction of the overscan columns
    tmp = imredbB1[ovs_d:ovs_u,:]

    #Runs through all the columns in the overscan row, averaging the value of the bias for each column
    #Whilst taking into account bad pixels which are set to NaN
    for i in range((effx)):
        prx[i] = np.nanmean(imredbB1[(ovs_d):ovs_u,i])
        if prx[i] > 40000:
            prx[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the columns
    meanprx = np.nanmean(prx)
    stdx = np.nanstd(prx)
    
    #Plots a histogram with the values of the mean in each of the columns to check for any remaining bad pixels
    #plt.figure(6)
    #plt.hist(prx,bins=100)
    
    #Replaces all values higher/lower than 2 standard deviations from the mean by NaN
    prx[prx > (meanprx + 2*stdx)] = np.nan
    prx[prx < (meanprx - 2*stdx)] = np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indx = np.isfinite(prx)
    
    #Calculates the parameters for the polyfit function
    px = np.polyfit(x[indx],prx[indx],5)
    
    #Builds the poly function automatically
    u = np.polyval(px,x)
    
    #Scatter so not to connect the dots; Plot to show the line;
    #Sets the color and the y-axis limit
    #plt.figure(7)
    #plt.scatter(x,prx)
    #plt.plot(x,u,color='red')
    #plt.ylim(meanprx-stdx,meanprx+stdx)
    
    #Transposes the template matrix to match dimensions
    tmp = np.transpose(tmp)

    #Plots the values of the bias counts in the overscan row
    #Sets the color and the y-axis limit
    #plt.figure(8)
    #plt.plot(x,tmp,color='red')
    #plt.ylim(7, 35)
    
    #The effective y-dimension of the Channel
    effy = ymaxB - yminB
    
    #Replicate the polyfit into an image of effx columns
    bias = np.tile(u,(effy,1))

    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelB = imredbB1[yminB:ymaxB,:]

    #Applies the overscan reduction due to the residual noise left over in the overscan rows
    imredbB2 = channelB - bias
    
    #Boundaries of overscan columns
    ovs_l,ovs_r = head['S_OSMN31'],head['S_OSMX31']
    
    #Creates an empty vector of dimension NAXIS2
    pry = np.empty(head['NAXIS2'])
    
    #Creates an integer vector from 0 to NAXIS2
    z = np.arange(head['NAXIS2'])
    
    #Runs through all the rows in the overscan column, averaging the value of the bias for each row
    #Whilst taking into account bad pixels which are set to NaN
    for i in range(head['NAXIS2']):
        pry[i] = np.nanmean(im[i,ovs_l:ovs_r]) #np.nanmean is always better than np.mean 'cause it accounts the NaNs
        if pry[i] > 60000:
            pry[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the rows
    meanpry = np.nanmean(pry)
    stdy = np.nanstd(pry)
    
    #Replaces all values higher/lower than 2 Stds from the mean by a NaN
    pry[pry > (meanpry + 2*stdy)]=np.nan
    pry[pry < (meanpry - 2*stdy)]=np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indy = np.isfinite(pry)
    
    #Calculates the parameters for the polyfit function
    py = np.polyfit(z[indy],pry[indy],5)
    
    #Builds the poly function automatically
    y = np.polyval(py,z)
    
    #Scatter so not to connect the dots; Plot to show the line; Show() is for when using the command line
    #plt.figure(9)
    #plt.scatter(z,pry)
    #plt.plot(z,y,color='red')
    #plt.ylim(meanpry-2*stdy,meanpry+2*stdy)
    
    #The effective x-dimension of the Channel
    effx = xmaxC - xminC
    
    #Replicate the polyfit into an image of effx columns
    bias = np.tile(y,(effx,1))
    
    #Bias must be transposed to match dimensions
    bias = np.transpose(bias)

    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelC = im[:,xminC:xmaxC]
    
    #Applies the overscan reduction due to the overscan columns
    imredbC1 = channelC - bias
    
    #Boundaries of overscan rows
    ovs_d,ovs_up = head['S_OSMN32'],head['S_OSMX32']
    
    #Offset due to some errors encountered
    ovs_u = ovs_up - 2

    #Creates an empty vector of dimension effx
    prx = np.empty(effx)
    
    #Creates an integer vector from xminC to xmaxC
    x = np.arange(xminC,xmaxC)
    
    #Template matrix for the overscan rows, which have already been affected
    #By the reduction of the overscan columns
    tmp = imredbC1[ovs_d:ovs_u,:]

    #Runs through all the columns in the overscan row, averaging the value of the bias for each column
    #Whilst taking into account bad pixels which are set to NaN
    for i in range((effx)):
        prx[i] = np.nanmean(imredbC1[(ovs_d):ovs_u,i])
        if prx[i] > 15000:
            prx[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the columns
    meanprx = np.nanmean(prx)
    stdx = np.nanstd(prx)

    #Plots a histogram with the values of the mean in each of the columns to check for any remaining bad pixels
    #plt.figure(10)
    #plt.hist(prx,bins=100)
    
    #Replaces all values higher/lower than a standard deviations from the mean by NaN
    prx[prx > (meanprx + stdx)] = np.nan
    prx[prx < (meanprx - stdx)] = np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indx = np.isfinite(prx)
    
    #Calculates the parameters for the polyfit function
    px = np.polyfit(x[indx],prx[indx],5)
    
    #Builds the poly function automatically
    u = np.polyval(px,x)
    
    #Scatter so not to connect the dots; Plot to show the line;
    #Sets the color and the y-axis limit
    #plt.figure(11)
    #plt.scatter(x,prx)
    #plt.plot(x,u,color='red')
    #plt.ylim(meanprx-stdx,meanprx+stdx)

    #Transposes the template matrix to match dimensions
    tmp = np.transpose(tmp)

    #Plots the values of the bias counts in the overscan row
    #Sets the color and the y-axis limit
    #plt.figure(12)
    #plt.plot(x,tmp,color='red')
    #plt.ylim(7, 35)
    
    #The effective y-dimension of the Channel
    effy = ymaxC - yminC
    
    #Replicate the polyfit into an image of effy columns
    bias = np.tile(u,(effy,1))

    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelC = imredbC1[yminC:ymaxC,:]

    #Applies the overscan reduction due to the residual noise left over in the overscan rows
    imredbC2 = channelC - bias

    #Boundaries of overscan columns
    ovs_l,ovs_r = head['S_OSMN41'],head['S_OSMX41']
    
    #Creates an empty vector of dimension NAXIS2
    pry = np.empty(head['NAXIS2']) #Creates an empty vector of dimension NAXIS2
    
    #Creates an integer vector from 0 to NAXIS2
    z = np.arange(head['NAXIS2']) #Creates an integer vector from 0 to NAXIS2
    
    #Runs through all the rows in the overscan column, averaging the value of the bias for each row
    #Whilst taking into account bad pixels which are set to NaN
    for i in range(head['NAXIS2']):
        pry[i] = np.nanmean(im[i,ovs_l:ovs_r]) #np.nanmean is always better than np.mean 'cause it accounts the NaNs
        if pry[i] > 60000:
            pry[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the rows
    meanpry = np.nanmean(pry)
    stdy = np.nanstd(pry)
    
    #Replaces all values higher/lower than 2 Stds from the mean by a NaN
    pry[pry > (meanpry + 2*stdy)] = np.nan
    pry[pry < (meanpry - 2*stdy)] = np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indy = np.isfinite(pry)
    
    #Calculates the parameters for the polyfit function
    py = np.polyfit(z[indy],pry[indy],5)
    
    #Builds the poly function automatically
    y = np.polyval(py,z)
    
    #Scatter so not to connect the dots; Plot to show the line;
    #plt.figure(13)
    #plt.scatter(z,pry)
    #plt.plot(z,y,color='red')
    #plt.ylim(meanpry-2*stdy,meanpry+2*stdy)
    
    #The effective x-dimension of the Channel
    effx = xmaxD - xminD
    
    #Replicate the polyfit into an image of effx columns
    bias = np.tile(y,(effx,1))

    #Bias must be transposed to match dimensions
    bias = np.transpose(bias)

    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelD = im[:,xminD:xmaxD]
    
    #Applies the overscan reduction due to the overscan columns
    imredbD1 = channelD - bias
    
    #Boundaries of overscan rows
    ovs_d,ovs_up = head['S_OSMN42'],head['S_OSMX42']
    
    #Offset due to some errors encountered
    ovs_u = ovs_up - 2

    #Creates an empty vector of dimension effx
    prx = np.empty(effx)
    
    #Creates an integer vector from xminD to xmaxD
    x = np.arange(xminD,xmaxD)
    
    #Template matrix for the overscan rows, which have already been affected
    #By the reduction of the overscan columns
    tmp = imredbD1[ovs_d:ovs_u,:]

    #Runs through all the columns in the overscan row, averaging the value of the bias for each column
    #Whilst taking into account bad pixels which are set to NaN
    for i in range((effx)):
        prx[i] = np.nanmean(imredbD1[(ovs_d):ovs_u,i])
        if prx[i] > 8000:
            prx[i] = np.nan
    
    #The mean and standard deviation for the array with the mean bias for all the columns
    meanprx = np.nanmean(prx)
    stdx = np.nanstd(prx)

    #Plots a histogram with the values of the mean in each of the columns to check for any remaining bad pixels
    #plt.figure(14)
    #plt.hist(prx,bins=100)
    
    #Replaces all values higher/lower than a standard deviations from the mean by NaN
    prx[prx>(meanprx + stdx)] = np.nan
    prx[prx<(meanprx - stdx)] = np.nan
    
    #Index that tells you which positions in this array are finite
    #The plot then only takes into account those pixels which are "true"
    indx = np.isfinite(prx)
    
    #Calculates the parameters for the polyfit function
    px = np.polyfit(x[indx],prx[indx],5)
    
    #Builds the poly function automatically
    u = np.polyval(px,x)
    
    #Scatter so not to connect the dots; Plot to show the line;
    #Sets the color and the y-axis limit
    #plt.figure(15)
    #plt.scatter(x,prx)
    #plt.plot(x,u,color='red')
    #plt.ylim(meanprx-stdx,meanprx+stdx)
    
    #Transposes the template matrix to match dimensions
    tmp = np.transpose(tmp)

    #Plots the values of the bias counts in the overscan row
    #Sets the color and the y-axis limit
    #plt.figure(16)
    #plt.plot(x,tmp,color='red')
    #plt.ylim(7, 35)
    
    #The effective y-dimension of the Channel
    effy = ymaxD - yminD
    
    #Replicate the polyfit into an image of effx columns
    bias = np.tile(u,(effy,1))
    
    #Creates a matrix with the same dimensions as im but only ranging from the edges of the overscan
    channelD = imredbD1[yminD:ymaxD,:]
    
    #Applies the overscan reduction due to the residual noise left over in the overscan rows
    imredbD2 = channelD - bias
    
    #Show() is for when using the command line or when we have multiple figures
    #plt.show()
    
    if head['DET-ID'] == 3 or head['DET-ID'] == 4 or head['DET-ID'] == 5 or head['DET-ID'] == 8 or head['DET-ID'] == 9:
        print('Hello!')
        imnew = np.concatenate([imredbD2,imredbC2,imredbB2,imredbA2],axis=1)
        print(head['DET-ID'])
    else:
        imnew = np.concatenate([imredbA2,imredbB2,imredbC2,imredbD2],axis=1)
    #hdu = fitsprint('Hello!').PrimaryHDU(data=imnew)
    #hdu.writeto(head['DATE-OBS']+'_'+head['UT-STR']+'_'+head['FILTER01']+'_'+head['DETECTOR']+'_'head['FRAMEID']+'imnew.fits',overwrite=True)
    
    return imnew

def flat(file):
    
    #The path where the dataset is located
    path = '/home/yobd/Astronomy/CrA_Subaru/raw_data/'
    
    #The path where the master flats are saved
    flatpath = '/home/yobd/Documents/FlatMasters/'
    
    #Encodes the path of the dataset to be used by other OS methods
    folder = os.fsencode(path)
    
    #Encodes the path of the master flats to be used by other OS methods
    flatfolder = os.fsencode(flatpath)
    
    #List of matrixes where the different reduced flatfield images will be stored
    temp = []
    
    #Opens the object image and registers its header for usage later when comparing date, filter and detector
    #For the flatfield images selection
    hdu = fits.open(file)
    head = hdu[0].header
    
    a = 0
    
    #Reduces the overscan in the object file
    im = func(file)
    
    #Checks the flat master folder for an already existing master image to be used during reduction
    #If you change the way flatfield masters are created, delete the existing master flats so it re-creates them
    for item in os.listdir(flatfolder):
        
        item = os.fsdecode(item)
        
        if item == head['DATE-OBS']+'_'+head['FILTER01']+'_'+head['DETECTOR']+'FLATMASTER.fits':
            
            item = flatpath + item
            hdumasterfound = fits.open(item)
            flatim = hdumasterfound[0].data
            
            #Applies the flatfield reduction to the object image
            imflatred = im / flatim
            
            a = 1
            
            break
    
    if a == 0:
        
        #Checks every file in the path of the dataset and searches for the domeflats corresponding to the object image
        for item in os.listdir(folder):
            
            item = os.fsdecode(item) #Decodes the current item so it becomes a string
            
            item = path+item #To comply with the func() function parameter
            
            #Opens and registers the hader of the current item file
            hduflat = fits.open(item) 
            headflat = hduflat[0].header
            
            #Checks if the current item is a domeflat image and if it corresponds to the same filter, detector
            #And was registered in the same date of observation as the object image we are flatfield reducing
            if headflat['DATA-TYP'] == 'DOMEFLAT' and headflat['DETECTOR'] == head['DETECTOR'] and headflat['FILTER01'] == head['FILTER01'] and headflat['DATE-OBS'] == head['DATE-OBS']:
                
                #Reduces the domeflat image
                imflat = func(item)
                
                #The median of the reduced domeflat image
                m = np.nanmedian(imflat)
                
                #By dividing the domeflat image with its median we obtain general values around unity
                #Throughout the image
                imflat = imflat/m
                
                #The processed domeflat image is added to the list of images initialized at the beginning of the function
                temp.append(imflat)
        
        #Stacks all the domeflat images on top of each other
        imtest = np.stack(temp,2)
        
        #Creates a median image of the stack of domeflats which was just initialized
        flatim = np.nanmedian(imtest,2)
        
        #Applies the flatfield reduction to the object image
        imflatred = im / flatim
    
        #Writes the final processed median domeflat image into a .fits file
        hduflatmaster = fits.PrimaryHDU(data=flatim)
        hduflatmaster.writeto(flatpath+head['DATE-OBS']+'_'+head['FILTER01']+'_'+head['DETECTOR']+'FLATMASTER.fits',overwrite=True)
    
    imfinal = bpix(imflatred, flatim)
    
    savepath = '/home/yobd/Documents/SciObj/'
    
    #Writes the flatfield reduced object image into a .fits file
    hduimfinal = fits.PrimaryHDU(data=imfinal,header=head)
    hduimfinal.writeto(savepath+head['DATE-OBS']+'_'+head['UT-STR']+'_'+head['FILTER01']+'_'+head['DETECTOR']+'_'+head['FRAMEID']+'imnew.fits',overwrite=True)
    
    return imfinal

def bpix(imflatred, flatim):
    im = imflatred
    bpixel = np.where((flatim < 0.5), 1, 0)
    i = 0
    j = 0
    for row in flatim:
        for columns in flatim.T:
            if bpixel[i][j] == 1:
                
                if (i == 0 and j == 0):
                    
                    x = np.array([[i+1,j], [i,j+1], [i+1,j+1]])
                
                    y = np.array([im[i+1][j], im[i][j+1], im[i+1][j+1]])
                    
                elif (i == 0 and j == 2043):
                    
                    x = np.array([[i+1,j], [i,j-1], [i+1,j-1]])
                
                    y = np.array([im[i+1][j], im[i][j-1], im[i+1][j-1]])
                
                elif i == 0:
                    
                    x = np.array([[i+1,j], [i,j-1], [i,j+1], [i+1,j-1], [i+1,j+1]])
                
                    y = np.array([im[i+1][j], im[i][j-1], im[i][j+1], im[i+1][j-1], im[i+1][j+1]])
                
                elif (i == 4175 and j == 0):
                    
                    x = np.array([[i-1,j], [i,j+1], [i-1,j+1]])
                
                    y = np.array([im[i-1][j], im[i][j+1], im[i-1][j+1]])
                
                elif (i == 4175 and j == 2043):
                    
                    x = np.array([[i-1,j], [i,j-1], [i-1,j-1]])
                
                    y = np.array([im[i-1][j], im[i][j-1], im[i-1][j-1]])
                    
                elif i == 4175:
                    
                    x = np.array([[i,j-1], [i,j+1], [i-1,j-1], [i-1,j+1], [i-1,j]])
                
                    y = np.array([im[i][j-1], im[i][j+1], im[i-1][j-1], im[i-1][j+1], im[i-1][j]])
                
                elif j == 0:
                    
                    x = np.array([[i,j+1], [i-1,j+1], [i-1,j], [i+1,j], [i+1,j+1]])
                
                    y = np.array([im[i][j+1], im[i-1][j+1], im[i-1][j], im[i+1][j], im[i+1][j+1]])
                
                elif j == 2043:
                    
                    x = np.array([[i,j-1], [i-1,j-1], [i-1,j], [i+1,j], [i+1,j-1]])
                
                    y = np.array([im[i][j-1], im[i-1][j-1], im[i-1][j], im[i+1][j], im[i+1][j-1]])
                
                else:
                
                    x = np.array([[i-1,j], [i,j-1], [i+1,j], [i,j+1], [i-1,j-1], [i+1,j-1], [i+1,j+1], [i-1,j+1]])
                
                    y = np.array([im[i-1][j], im[i][j-1], im[i+1][j], im[i][j+1], im[i-1][j-1], im[i+1][j-1], im[i+1][j+1], im[i-1][j+1]])
                
                im[i][j] = interpolate.NearestNDInterpolator(x,y)(i,j)
                
                
            j = j + 1
        i = i + 1
        j = 0
    
    return im